In [16]:
import io
import os
import json
import base64
from openai import OpenAI
from attributes_predictor_module.openai.schemas import GarmentDescription
from dotenv import load_dotenv
from PIL import ImageOps, Image
load_dotenv()

def encode_image_pil(pil_image):
    buffered = io.BytesIO()
    pil_image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def resize_image_to_fit_larger_side(img, target_size):
    original_width, original_height = img.size
    if original_width > original_height:
        new_width = target_size
        new_height = int((target_size / original_width) * original_height)
    else:
        new_height = target_size
        new_width = int((target_size / original_height) * original_width)

    resized_image = img.resize((new_width, new_height), Image.LANCZOS)
    return resized_image

def form_image_request_body(prompts, image):
    base64_image = encode_image_pil(image)
    request_body = [{
            "role": "system",
            "content": prompts["system"]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompts["user"]},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]
        }
    ]
    return request_body

client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
)

# current_file_directory = os.path.dirname(os.path.abspath(__file__))
# relative_path = os.path.join(current_file_directory, 'config.json')
relative_path = "/workspace/VTO_demo/attributes_predictor_module/openai/config.json"
with open(relative_path, 'r') as f:
    config = json.load(f)

def get_garment_description(image):
    image = resize_image_to_fit_larger_side(image, config['image_size'])
    messages = form_image_request_body(config['prompts'], image)
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=GarmentDescription,
    )
    return response.choices[0].message.parsed

In [15]:
image = Image.open('example/cloth/Long Sleeve Belted Wrap Coat in Dark Green with Stand Collar.jpg')

In [17]:
response = get_garment_description(image)